Link to colab: https://colab.research.google.com/drive/1zla80nWqRsYxsYsTAR3tTYxjdyqwBSco?usp=sharing

In [ ]:
!wget http://www.computerchess.org.uk/ccrl/4040/CCRL-4040.[1589759].pgn.7z

--2023-05-08 09:32:42--  http://www.computerchess.org.uk/ccrl/4040/CCRL-4040.[1589759].pgn.7z
Resolving www.computerchess.org.uk (www.computerchess.org.uk)... 185.45.66.155
Connecting to www.computerchess.org.uk (www.computerchess.org.uk)|185.45.66.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308356584 (294M) [application/x-7z-compressed]
Saving to: ‘CCRL-4040.[1589759].pgn.7z’

CCRL-4040.[1589759] 100%[===================>] 294.07M  11.3MB/s    in 29s     

2023-05-08 09:33:12 (10.3 MB/s) - ‘CCRL-4040.[1589759].pgn.7z’ saved [308356584/308356584]



In [ ]:
!7z x CCRL-4040.[1589759].pgn.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 308356584 bytes (295 MiB)

Extracting archive: CCRL-4040.[1589759].pgn.7z
--
Path = CCRL-4040.[1589759].pgn.7z
Type = 7z
Physical Size = 308356584
Headers Size = 154
Method = PPMD
Solid = -
Blocks = 1

  0%      0% - CCRL-4040.[1589759].pgn                                1% - CCRL-4040.[1589759].pgn                                2% - CCRL-4040.[1589759].pgn                                3% - CCRL-4040.[1589759].pgn                                4% - CCRL-4040.[1589759].

In [ ]:
!pip install chess

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
import chess.pgn
import numpy as np
import pandas as pd

In [ ]:
# Rationale for the retarded way of reading PGN's:
# Well, reading 1.5mln games is slow, that's all - this way I'm done in few minutes instead of ~6 hours (pandas is fast)

df = pd.read_csv('CCRL-4040.[1589759].pgn', names=['data'], sep='|')
# Separator set to '|', one that isn't used anywhere to keep single column only
df = df[
    (~df['data'].str.startswith('['))
    | (df['data'].str.startswith('[Result'))
    | (df['data'].str.startswith('[PlyCou'))
    | (df['data'].str.startswith('[WhiteE'))
    | (df['data'].str.startswith('[BlackE'))
].reset_index(drop=True)

df

,data
0,"[Result ""1-0""]"
1,"[PlyCount ""83""]"
2,"[WhiteElo ""2634""]"
3,"[BlackElo ""2743""]"
4,1. d4 Nf6 2. c4 g6 3. Nc3 d5 4. cxd5 Nxd5 5. e...
...,...
23503257,90. Ke7 Qb7+ 91. Kf6 Qg7+ 92. Ke6 Qa7 93. Qd2 ...
23503258,Qc3 Qb6+ 97. Ke7 Qb7+ 98. Ke6 Kg8 99. Qd2 Qf7+...
23503259,Qa2+ Kg7 103. Qe6 Qc5 104. Qb3 Qd4+ 105. Ke8 Q...
23503260,108. Qc3+ Kh7 109. Qb3 Qg4+ 110. Kc6 Qc8+ 111....


In [ ]:
df_work = df.copy()
df_work['data_c'] = df_work['data']
df_work['meta'] = df_work['data'].str.startswith('[')
df_work['data'] = df_work['data'].where(df_work['meta'] == True, '')
df_work['data_c'] = df_work['data_c'].where(df_work['meta'] == False, '')
df_work['data_c'] = df_work['data_c'].apply(lambda x: ' '.join(filter(lambda x: not x.endswith('.'), x.split(' '))))

df_work

,data,data_c,meta
0,"[Result ""1-0""]",,True
1,"[PlyCount ""83""]",,True
2,"[WhiteElo ""2634""]",,True
3,"[BlackElo ""2743""]",,True
4,,d4 Nf6 c4 g6 Nc3 d5 cxd5 Nxd5 e4 Nxc3 bxc3 Bg7...,False
...,...,...,...
23503257,,Ke7 Qb7+ Kf6 Qg7+ Ke6 Qa7 Qd2 Kg7 Qc3+ Kg8 Qc8...,False
23503258,,Qc3 Qb6+ Ke7 Qb7+ Ke6 Kg8 Qd2 Qf7+ Kd6 Qf6+ Kd...,False
23503259,,Qa2+ Kg7 Qe6 Qc5 Qb3 Qd4+ Ke8 Qe5+ Kd7 f4 gxf4...,False
23503260,,Qc3+ Kh7 Qb3 Qg4+ Kc6 Qc8+ Kd6 Qf8+ Kd7 Qf4 Qd5,False


In [ ]:
results = []

for d1, d2, is_meta in df_work.values:
    if is_meta:
        results += [d1]
        tmp = False
    else:
        if not tmp:
            results += [d2]
            tmp = True
        else:
            results[-1] = ' '.join([results[-1], d2])

In [ ]:
df_clean = pd.DataFrame(results, columns=['data'])
df_clean['game'] = df_clean['data'].where(~df_clean['data'].str.startswith('['), '')
df_clean['data'] = df_clean['data'].str.strip('[]')
df_clean['result'] = df_clean['data'].where(df_clean['data'].str.startswith('Result'), '')
df_clean['len'] = df_clean['data'].where(df_clean['data'].str.startswith('PlyCount'), '')
df_clean['welo'] = df_clean['data'].where(df_clean['data'].str.startswith('WhiteElo'), '')
df_clean['belo'] = df_clean['data'].where(df_clean['data'].str.startswith('BlackElo'), '')

df_clean = df_clean.drop('data', axis='columns')
df_clean['result'] = df_clean['result'].apply(lambda x: x.split(' ')[-1].strip('"'))
df_clean['len'] = df_clean['len'].apply(lambda x: x.split(' ')[-1].strip('"'))
df_clean['welo'] = df_clean['welo'].apply(lambda x: x.split(' ')[-1].strip('"'))
df_clean['belo'] = df_clean['belo'].apply(lambda x: x.split(' ')[-1].strip('"'))

df_clean

,game,result,len,welo,belo
0,,1-0,,,
1,,,83,,
2,,,,2634,
3,,,,,2743
4,d4 Nf6 c4 g6 Nc3 d5 cxd5 Nxd5 e4 Nxc3 bxc3 Bg7...,,,,
...,...,...,...,...,...
7948329,,1/2-1/2,,,
7948330,,,234,,
7948331,,,,2222,
7948332,,,,,2330


In [ ]:
example = {'result': '', 'len': '', 'welo': '', 'belo': '', 'game': ''}
games = []

for game, result, len, welo, belo in df_clean.values:
    if result != '':
        example['result'] = result
    if len != '':
        example['len'] = len
    if welo != '':
        example['welo'] = welo
    if belo != '':
        example['belo'] = belo
    elif game != '':
        example['game'] = game
        games += [example]
        example = {'result': '', 'len': '', 'welo': '', 'belo': '', 'game': ''}

games = pd.DataFrame(games)
assert games.shape[0] == 1589759, "Number of games different than expected"

games

,result,len,welo,belo,game
0,1-0,83,2634,2743,d4 Nf6 c4 g6 Nc3 d5 cxd5 Nxd5 e4 Nxc3 bxc3 Bg7...
1,0-1,150,2743,2634,d4 Nf6 c4 g6 Nc3 d5 cxd5 Nxd5 e4 Nxc3 bxc3 Bg7...
2,1-0,175,2634,2743,Nf3 d5 d4 Nf6 c4 c6 Nc3 dxc4 a4 Bf5 e3 e6 Bxc4...
3,1/2-1/2,185,2743,2634,Nf3 d5 d4 Nf6 c4 c6 Nc3 dxc4 a4 Bf5 e3 e6 Bxc4...
4,0-1,138,2634,2743,e4 e6 d4 d5 Nd2 Nf6 e5 Nfd7 Bd3 c5 c3 Nc6 Ne2 ...
...,...,...,...,...,...
1589754,0-1,79,2235,2330,d4 d5 c4 e6 Nf3 Nf6 g3 Bb4+ Bd2 Be7 Bg2 O-O O-...
1589755,1-0,57,2330,2276,d4 Nf6 c4 e6 Nf3 Bb4+ Bd2 a5 Nc3 b6 Bg5 Bb7 e3...
1589756,0-1,109,2276,2330,d4 Nf6 c4 e6 Nf3 Bb4+ Bd2 a5 Nc3 b6 Bg5 Bb7 e3...
1589757,1-0,92,2330,2222,d4 Nf6 c4 e6 Nf3 Bb4+ Bd2 Qe7 g3 Nc6 Nc3 Bxc3 ...


In [ ]:
print("Data quality")

print(f"Empty result: {(games['result'] == '').sum()}")
print(f"Empty len:    {(games['len'] == '').sum()}")
print(f"Empty welo:   {(games['welo'] == '').sum()}")
print(f"Empty belo:   {(games['belo'] == '').sum()}")
print(f"Empty game:   {(games['game'] == '').sum()}")

print()
display(games.dtypes)

Data quality
Empty result: 0
Empty len:    0
Empty welo:   229
Empty belo:   232
Empty game:   0



result    object
len       object
welo      object
belo      object
game      object
dtype: object

In [ ]:
games['result'] = games['result'].apply(lambda x: {'1-0': 'W', '1/2-1/2': 'D', '0-1': 'B'}[x])
games = games[games['welo'] != '']
games = games[games['belo'] != '']
games = games.astype({'len': np.int32, 'welo': np.int32, 'belo': np.int32})
games = games.sample(frac=1).reset_index(drop=True)

games

,result,len,welo,belo,game
0,D,64,3388,3248,e4 c6 d4 d5 e5 Bf5 Nf3 e6 c3 Nd7 Be2 f6 O-O fx...
1,W,144,3249,3112,e4 c5 Nf3 Nc6 Bb5 g6 Bxc6 dxc6 d3 Bg7 h3 Nf6 N...
2,D,122,3188,3241,e4 e5 Nf3 Nc6 Bb5 a6 Ba4 Nf6 O-O Be7 Re1 b5 Bb...
3,W,149,3069,3054,e4 e5 Nf3 Nc6 Bb5 a6 Ba4 Nf6 O-O Be7 Re1 b5 Bb...
4,D,98,3457,3446,d4 d5 c4 dxc4 Nf3 Nf6 e3 e6 Bxc4 c5 O-O a6 a4 ...
...,...,...,...,...,...
1589300,W,130,3008,2983,d4 g6 c4 Bg7 g3 d6 Bg2 Nd7 Nf3 Ngf6 Nc3 O-O O-...
1589301,D,122,3509,3427,Nf3 d5 c4 d4 e3 Nc6 b4 dxe3 fxe3 Nxb4 d4 e5 a3...
1589302,W,105,2613,2630,b3 Nf6 Bb2 g6 e3 Bg7 Nf3 O-O d4 d6 Be2 Nc6 O-O...
1589303,W,157,2632,2668,d4 Nf6 c4 e6 Nf3 b6 g3 Ba6 b3 Bb7 Bg2 Bb4+ Bd2...


In [ ]:
print("Results rates:")
display(games['result'].value_counts())
games.describe().T

Results rates:


D    697323
W    520664
B    371318
Name: result, dtype: int64

,count,mean,std,min,25%,50%,75%,max
len,1589305.0,132.496391,52.054677,28.0,100.0,123.0,152.0,750.0
welo,1589305.0,2847.328488,390.900033,1379.0,2597.0,2886.0,3138.0,3534.0
belo,1589305.0,2847.311651,390.899431,1379.0,2597.0,2886.0,3138.0,3534.0


In [ ]:
games_filt = games[np.minimum(games['welo'], games['belo']) >= 3000].reset_index(drop=True)

games_filt

,result,len,welo,belo,game
0,D,64,3388,3248,e4 c6 d4 d5 e5 Bf5 Nf3 e6 c3 Nd7 Be2 f6 O-O fx...
1,W,144,3249,3112,e4 c5 Nf3 Nc6 Bb5 g6 Bxc6 dxc6 d3 Bg7 h3 Nf6 N...
2,D,122,3188,3241,e4 e5 Nf3 Nc6 Bb5 a6 Ba4 Nf6 O-O Be7 Re1 b5 Bb...
3,W,149,3069,3054,e4 e5 Nf3 Nc6 Bb5 a6 Ba4 Nf6 O-O Be7 Re1 b5 Bb...
4,D,98,3457,3446,d4 d5 c4 dxc4 Nf3 Nf6 e3 e6 Bxc4 c5 O-O a6 a4 ...
...,...,...,...,...,...
576215,W,88,3168,3092,d4 Nf6 c4 e6 Nc3 Bb4 Qc2 O-O a3 Bxc3+ Qxc3 b6 ...
576216,B,102,3192,3249,c4 e6 g3 Nf6 Bg2 d5 Nf3 Nbd7 d4 dxc4 Qa4 a6 Qx...
576217,D,123,3184,3136,c4 Nf6 g3 c6 Bg2 d5 Nf3 Bf5 Qb3 Qb6 cxd5 Qxb3 ...
576218,W,168,3195,3060,c4 e5 g3 c6 b3 Nf6 Bb2 d6 Bg2 Be7 Nf3 Na6 Nc3 ...


In [ ]:
print("Results rates:")
display(games_filt['result'].value_counts())
games_filt.describe().T

Results rates:


D    337671
W    152354
B     86195
Name: result, dtype: int64

,count,mean,std,min,25%,50%,75%,max
len,576220.0,131.329614,49.038083,28.0,102.0,123.0,149.0,705.0
welo,576220.0,3236.511173,140.872714,3000.0,3112.0,3230.0,3348.0,3534.0
belo,576220.0,3236.501630,140.871551,3000.0,3112.0,3230.0,3348.0,3534.0


In [ ]:
games_filt['game'] = games_filt['game'].apply(lambda x:
    ' '.join(map(lambda x: f"{['W', 'B'][x[0] % 2]}{(x[0] + 2) // 2}.{x[1]}", enumerate(x.split(' ')[:-1]))))

games_filt

,result,len,welo,belo,game
0,D,64,3388,3248,W1.e4 B1.c6 W2.d4 B2.d5 W3.e5 B3.Bf5 W4.Nf3 B4...
1,W,144,3249,3112,W1.e4 B1.c5 W2.Nf3 B2.Nc6 W3.Bb5 B3.g6 W4.Bxc6...
2,D,122,3188,3241,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bb5 B3.a6 W4.Ba4 ...
3,W,149,3069,3054,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bb5 B3.a6 W4.Ba4 ...
4,D,98,3457,3446,W1.d4 B1.d5 W2.c4 B2.dxc4 W3.Nf3 B3.Nf6 W4.e3 ...
...,...,...,...,...,...
576215,W,88,3168,3092,W1.d4 B1.Nf6 W2.c4 B2.e6 W3.Nc3 B3.Bb4 W4.Qc2 ...
576216,B,102,3192,3249,W1.c4 B1.e6 W2.g3 B2.Nf6 W3.Bg2 B3.d5 W4.Nf3 B...
576217,D,123,3184,3136,W1.c4 B1.Nf6 W2.g3 B2.c6 W3.Bg2 B3.d5 W4.Nf3 B...
576218,W,168,3195,3060,W1.c4 B1.e5 W2.g3 B2.c6 W3.b3 B3.Nf6 W4.Bb2 B4...


In [ ]:
games_filt.to_parquet("chess_data_ccrl.parquet")

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

!cp chess_data_ccrl.parquet "/content/drive/MyDrive/Praca magisterska/Data"

drive.flush_and_unmount()

Mounted at /content/drive/
